<a href="https://colab.research.google.com/github/AmelxJamal/NLP_Week1_Lab1/blob/main/Amel_Abdelraheem_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this second part of the lab, we will implement a language identifier trained on the same data, but using Logistic Regression instead of Naive Bayes.

In [1]:
import io, sys, math
import numpy as np
from collections import defaultdict

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


This function is used to build the dictionary, or vocabulary, which is a mapping from strings (or words) to integers (or indices). This will allow to build vector representations of documents. 

In [3]:
def build_dict(filename, threshold=1):
    fin = io.open(filename, 'r', encoding='utf-8')
    word_dict, label_dict = {}, {}
    counts = defaultdict(lambda: 0)
    for line in fin:
        tokens = line.split()
        label = tokens[0]

        if not label in label_dict:
            label_dict[label] = len(label_dict)

        for w in tokens[1:]:
            counts[w] += 1
            
    for k, v in counts.items():
        if v > threshold:
            word_dict[k] = len(word_dict)
    return word_dict, label_dict

This function is used to load the training dataset, and build vector representations of the training examples. In particular, a document or sentence is represented as a bag of words. Each example correspond to a sparse vector ` x` of dimension `V`, where `V` is the size of the vocabulary. The element `j` of the vector `x` is the number of times the word `j` appears in the document.

In [4]:
def load_data(filename, word_dict, label_dict):
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    dim = len(word_dict)
    for line in fin:
        tokens = line.split()
        label = tokens[0]

        yi = label_dict[label]
        xi = np.zeros(dim)
        for word in tokens[1:]:
            if word in word_dict:
                wid = word_dict[word]
                xi[wid] += 1.0
        data.append((yi, xi))
    return data

First, let's implement the softmax function. Don't forget numerical stability!

In [5]:
def softmax(x):
    ### FILL CODE
    
    x_normalized = x - max(x)
    log_softmax =  x_normalized - np.log(np.sum(np.exp(x_normalized), axis=0))

    return np.exp(log_softmax)

Now, let's implement the main training loop, by using stochastic gradient descent. The function will iterate over the examples of the training set. For each example, we will first compute the loss, before computing the gradient and performing the update.

In [6]:
def sgd(w, data, niter):
    nlabels, dim = w.shape
    
    for iter in range(niter):
        ### FILL CODE
        
        for x in data:

          label = x[0]
          value = x[1]
          
          proba = softmax(w@value)
          true_label = np.zeros(len(proba))
          true_label[label] = 1
         
          y_pred = np.argmax(proba)
          y_pred_vector = np.zeros(len(proba))
          y_pred_vector[y_pred] = 1
          
          loss = - np.log(proba[label])
          
          grad = ((y_pred_vector-true_label).reshape(nlabels,1)*value.reshape(1,dim))

          w = w - grad


        print('iteration',iter, ': loss ', loss)
          

    return w

The next function will predict the most probable label corresponding to example `x`, given the trained classifier `w`.

In [7]:
def predict(w, x):
    ## FILL CODE
    proba = softmax(x@w.T)
    y_pred = np.argmax(proba)
    
    return y_pred

Finally, this function will compute the accuracy of a trained classifier `w` on a validation set.

In [8]:
def compute_accuracy(w, valid_data):
    ## FILL CODE
    score = 0
    for x in valid_data:
      label = x[0]
      value = x[1]
      y_pred = predict(w, value)
      if y_pred == label:
        score +=1
      
    accuracy = score/len(valid_data)
    return accuracy * 100

In [9]:
print("")
print("** Logistic Regression **")
print("")

word_dict, label_dict = build_dict("/content/drive/MyDrive/NLP_Week1_Lab1/train1.txt")
train_data = load_data("/content/drive/MyDrive/NLP_Week1_Lab1/train1.txt", word_dict, label_dict)
valid_data = load_data("/content/drive/MyDrive/NLP_Week1_Lab1/valid1.txt", word_dict, label_dict)

nlabels = len(label_dict)
dim = len(word_dict)
w = np.zeros([nlabels, dim])
w = sgd(w, train_data, 5)
print("")
print("Validation accuracy: %.3f" % compute_accuracy(w, valid_data))
print("")


** Logistic Regression **

iteration 0 : loss  1.4056975980350919
iteration 1 : loss  1.4056975980350919
iteration 2 : loss  1.4056975980350919
iteration 3 : loss  1.4056975980350919
iteration 4 : loss  1.4056975980350919

Validation accuracy: 90.300



In [ ]:
train_data[0][1].shape, w.shape

((5826,), (10, 5826))